## Initialize Input Files for IPMOF (initialize_IPMOF.py)

### Initialize necessary information

1. Read FF_Parameters excel file to get force field parameters for atoms
2. Initialize force field selection, cutOff radius and grid size for energy map

### Create input files for MOF files in a given directory
1. Reaf MOF files in ".mol2" format from given directory and create a list
2. Choose the first MOF1 as the stationary (map) MOF, initialize its variables, and calculate packed coordinates
3. For all the MOFs in the list:
4. Initialize MOF2 variables for the mobile MOF
5. Calculate energy map
6. Export energy map -> .json
7. Export MOF2 structure information -> .json

In [1]:
import math
import os
os.chdir('C:\\Kutay\\IPMOF')
from energyMapFunctions import *

# Read excel file containing force field information
excelFileDir = 'C:\\Users\\kutay\\iPython\\IPMOF\\FF_Parameters.xlsx'
UFF = readFFparameters(excelFileDir, 'UFF')

# Read radius list from the excel file
radiusFileDir = 'C:\\Users\\kutay\\Dropbox\\Academic\\WilmerLab\\Research\\IPMOF\\IPMOFvisualization\\atomicRadius.xlsx'
radiusList = readAtomRadius(radiusFileDir)

In [2]:
# Generate MOF list
expMOFs_dir = r'C:\Kutay\MOFs\IPMOF_Experimental'
expMOFs = os.listdir(expMOFs_dir)

heteroIPMOFs = []
for expMOF in expMOFs:
    heteroIPMOF_dir = os.path.join(expMOFs_dir, expMOF)
    heteroIPMOFs.append(generateMOFlist(heteroIPMOF_dir, '.mol2'))
    print(heteroIPMOFs[-1])

['COGCOS-P1.mol2', 'COGCOS-P2.mol2']
['ETAXAS-P1.mol2', 'ETAXAS-P2.mol2']
['MAHYID-P1.mol2', 'MAHYID-P2.mol2']
['PELQII-P1.mol2', 'PELQII-P2.mol2']
['TIVYED-P1.mol2', 'TIVYED-P2.mol2']
['UNAZIT-P1.mol2', 'UNAZIT-P2.mol2']
['WEBZEK-P1.mol2', 'WEBZEK-P2.mol2']


In [25]:
MOFindex = 6
IPMOF_list = heteroIPMOFs[MOFindex]

# Read structure files, create MOF class and initialize variables for two given MOFs 
MOFs = []
table = []
for mofCount, mof in enumerate(IPMOF_list):
    newMOF = MOF()
    newMOF.dir = os.path.join(expMOFs_dir, expMOFs[MOFindex], mof)
    newMOF.file = open(newMOF.dir, 'r')
    newMOF.initialize()
    newMOF.initializeFF(UFF)
    newMOF.UCV = ucv(newMOF)

    # Calculate cut-off radius as Rc = L/2
    widthA = newMOF.UCV / (newMOF.UCsize[1]*newMOF.UCsize[2] / math.sin(math.radians(newMOF.UCangle[0])))
    widthB = newMOF.UCV / (newMOF.UCsize[0]*newMOF.UCsize[2] / math.sin(math.radians(newMOF.UCangle[1])))
    widthC = newMOF.UCV / (newMOF.UCsize[0]*newMOF.UCsize[1] / math.sin(math.radians(newMOF.UCangle[2])))
    newMOF.cutOff = min(widthA/2, widthB/2, widthC/2)
    
    # Calculate packing for base MOF
    newMOF.packingFactor = Packing.factor(newMOF.UCsize, newMOF.cutOff)
    translationVectors, UCvectors = Packing.vectors(newMOF.packingFactor, newMOF.UCsize, newMOF.UCangle)
    newMOF.packedCoor = Packing.UC(translationVectors, newMOF.packingFactor, UCvectors, newMOF.atomCoor)
    newMOF.edgePoints = Packing.edgePoints(UCvectors)
    
    newMOF.name = mof.split('.')[0].split('_')[0]
    MOFs.append(newMOF)
    table.append([])
    table[mofCount].append(newMOF.name)
    table[mofCount].append(newMOF.uniqueAtomNames)
    table[mofCount].append(newMOF.UCsize)
    table[mofCount].append(newMOF.UCangle)
    table[mofCount].append(round(newMOF.cutOff, 2))
    #table[mofCount].append(newMOF.packingFactor)
        
from tabulate import tabulate

headers = ["MOF Name", "Atoms", "UCsize", "UCangle", "Cut-Off"]
print(tabulate(table, headers))

MOF Name    Atoms                  UCsize                  UCangle               Cut-Off
----------  ---------------------  ----------------------  ------------------  ---------
WEBZEK-P1   ['C', 'H', 'Cd', 'O']  [22.18, 22.18, 43.666]  [90.0, 90.0, 90.0]      11.09
WEBZEK-P2   ['Cd', 'C', 'H', 'O']  [22.18, 22.18, 43.666]  [90.0, 90.0, 90.0]      11.09


In [26]:
i = 0
j = 1
# Directory and format to export energy map and MOF structure files
exportFolder = r'C:\Kutay\IPMOF_Experimental\Input'
exportFormat = '.js'

# Creates atom list for non-repeating atoms from a list of MOF classes
MOFlist = [MOFs[j]]
atomList = getUniqueAtomList(MOFlist)

# Creates atom list required in the energy map of a combination of MOFs
eMapAtomList = getEnergyMapAtomList(MOFlist, atomList) 
           
# Create export directory for the base MOF and create folder
exportMOFfolder = os.path.join(exportFolder, MOFs[i].name)
os.mkdir(exportMOFfolder)

In [27]:
# --------------------- Calculate Energy Map -------------------
# For each MOF energy map is calculated with an atom list that consists of all the unique atoms z
# for a given list of MOFs
eMap = energyMap(MOFs[i], atomList, MOFs[i].cutOff, 1)

#avgEnergyLimits = calculateEnergyLimits(i, MOFlist, eMapAtomList, eMap)
energyLimit = 3E10
        
# Create export directory for the mobile MOF and create folder
exportMOFfolder2 = os.path.join(exportMOFfolder, MOFs[j].name)
os.mkdir(exportMOFfolder2)

# Create export directory for mobile MOF
exportDir_MOF = os.path.join(exportMOFfolder2, MOFs[j].name + exportFormat)
exportGLOBALMOFjs(MOFs[j], exportDir_MOF)
print(exportDir_MOF.split("\\")[-1] + ' is exported')

# Create export directory for energy map
exportDir_eMap = os.path.join(exportMOFfolder2, MOFs[i].name + '+' + MOFs[j].name + '_eMap' + exportFormat)
exportGLOBALEnergyMapjs(eMap, energyLimit, eMapAtomList, i, exportDir_eMap)
print(exportDir_eMap.split("\\")[-1] + ' is exported')

# Export base MOF structure
exportDir_baseMOF = os.path.join(exportMOFfolder2, 'base' + MOFs[i].name + exportFormat)
exportGLOBALBaseMOFjs(MOFs[i], exportDir_baseMOF)
print(exportDir_baseMOF.split("\\")[-1] + ' is exported')

WEBZEK-P2.js is exported
WEBZEK-P1+WEBZEK-P2_eMap.js is exported
baseWEBZEK-P1.js is exported


In [33]:
import math
coor = [-25, 21, 34]
UCsize = [6, 16, 8]
UCangle = [90, 90, 90]


def car2frac(coor, UCsize, UCangle, UCV):
    
    alp = UCangle[0] / 180 * math.pi
    bet = UCangle[1] / 180 * math.pi
    gam = UCangle[2] / 180 * math.pi
    
    a = UCsize[0]
    b = UCsize[1]
    c = UCsize[2]
    
    x = coor[0]
    y = coor[1]
    z = coor[2]
    
    v = math.sqrt(1 - math.cos(alp)**2 - math.cos(bet)**2 - math.cos(gam)**2 + 2*math.cos(alp)*math.cos(bet)*math.cos(gam))
    
    xfrac = 1/a*x 
    xfrac += - math.cos(gam)/(a*math.sin(gam))*y 
    xfrac += (math.cos(alp)*math.cos(gam)-math.cos(bet))/(a*v*math.sin(gam))*z
    
    yfrac = 0
    yfrac += 1/(b*math.sin(gam))*y
    yfrac += (math.cos(bet)*math.cos(gam)-math.cos(alp))/(b*v*math.sin(gam))*z
    
    zfrac = 0
    zfrac += 0
    zfrac += math.sin(gam)/(c*v)*z
    
    return [xfrac, yfrac, zfrac]

def fracPBC(fracCoor):
    pbcCoor = []
    for coor in fracCoor:
        pbcCoor.append(coor - math.floor(coor))
    return pbcCoor


fracCoor = car2frac(coor, UCsize, UCangle, ucv)
print(fracCoor)
pbcCoor = fracPBC(fracCoor)
print(pbcCoor)

[-4.166666666666666, 1.3124999999999998, 4.25]
[0.8333333333333339, 0.3124999999999998, 0.25]
